# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import numpy as np
import psycopg2
import pandas as pd
from sql_queries import *
from psycopg2.extensions import register_adapter

In [2]:
# Delete and Create all tables before re-running this ETL experiment
import create_tables
create_tables.main()

In [3]:
# Import helper functions from utils module
from utils import (addapt_numpy_float64,
                   addapt_numpy_int64,
                   get_files,
                   build_df_from_multiple_filepaths)

# numpy datatypes registered into psycopg2
register_adapter(np.float64, addapt_numpy_float64)
register_adapter(np.int64, addapt_numpy_int64)

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = (get_files(filepath='data/song_data'))
song_files[:10]

['/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/C/TRAACTB12903CAAF15.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/C/TRAACER128F4290F96.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/C/TRAACSL128F93462F4.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/C/TRAACNS128F14A2DF5.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/C/TRAACQT128F9331780.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/C/TRAACPE128F421C1B9.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/C/TRAACOW128F933E35F.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/C/TRAACCG128F92E8A55.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/song_data/A/A/C/TRAACIW12903CC0F6D.json',
 '/home/wb/gitrepos/udacity_project_data_model

In [6]:
df_songs = build_df_from_multiple_filepaths(filepaths=song_files)
df_songs.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR0RCMP1187FB3F427,30.08615,-94.10158,"Beaumont, TX",Billie Jo Spears,SOGXHEG12AB018653E,It Makes No Difference Now,133.32853,1992
1,1,ARMAC4T1187FB3FA4C,40.82624,-74.47995,"Morris Plains, NJ",The Dillinger Escape Plan,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,207.77751,2004
2,1,ARAJPHH1187FB5566A,40.70380,-73.83168,"Queens, NY",The Shangri-Las,SOYTPEP12AB0180E7B,Twist and Shout,164.80608,1964
3,1,AROUOZZ1187B9ABE51,40.79195,-73.94512,"New York, NY [Spanish Harlem]",Willie Bobo,SOBZBAZ12A6D4F8742,Spanish Grease,168.25424,1997
4,1,AR1Y2PT1187FB5B9CE,27.94017,-82.32547,Brandon,John Wesley,SOLLHMX12AB01846DC,The Emperor Falls,484.62322,0


In [7]:
df_songs.shape

(71, 10)

## #1: `artists` Table
---
This table must be created first since it has no foreign key dependency.

#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [8]:
df_songs["artist_id"].nunique()

69

In [9]:
df_artist = df_songs[["artist_id",
                      "artist_name",
                      "artist_location",
                      "artist_latitude",
                      "artist_longitude"]]\
                .dropna(axis=0, subset=["artist_id"])

df_artist.head()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR0RCMP1187FB3F427,Billie Jo Spears,"Beaumont, TX",30.08615,-94.10158
1,ARMAC4T1187FB3FA4C,The Dillinger Escape Plan,"Morris Plains, NJ",40.82624,-74.47995
2,ARAJPHH1187FB5566A,The Shangri-Las,"Queens, NY",40.70380,-73.83168
3,AROUOZZ1187B9ABE51,Willie Bobo,"New York, NY [Spanish Harlem]",40.79195,-73.94512
4,AR1Y2PT1187FB5B9CE,John Wesley,Brandon,27.94017,-82.32547


In [10]:
df_artist.shape

(71, 5)

In [11]:
df_songs["artist_id"].nunique()

69

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
for i, row in df_artist.iterrows():
    cur.execute(artist_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [13]:
df_songs["song_id"].nunique()

71

In [14]:
df_songs_table = df_songs[["song_id",
                           "title",
                           "artist_id",
                           "year",
                           "duration"]]\
                .dropna(axis=0, subset=["song_id"])

df_songs_table.head()

,song_id,title,artist_id,year,duration
0,SOGXHEG12AB018653E,It Makes No Difference Now,AR0RCMP1187FB3F427,1992,133.32853
1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,ARMAC4T1187FB3FA4C,2004,207.77751
2,SOYTPEP12AB0180E7B,Twist and Shout,ARAJPHH1187FB5566A,1964,164.80608
3,SOBZBAZ12A6D4F8742,Spanish Grease,AROUOZZ1187B9ABE51,1997,168.25424
4,SOLLHMX12AB01846DC,The Emperor Falls,AR1Y2PT1187FB5B9CE,0,484.62322


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [16]:
for i, row in df_songs_table.iterrows():
    cur.execute(song_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [17]:
log_files_paths = get_files(filepath='data/log_data')
log_files_paths[:5]

['/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-13-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-12-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-17-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-14-events.json',
 '/home/wb/gitrepos/udacity_project_data_modeling_postgres/data/log_data/2018/11/2018-11-11-events.json']

In [18]:
df_log = build_df_from_multiple_filepaths(filepaths=log_files_paths)
df_log.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Kevin,M,0,Arellano,None,free,"Harrisburg-Carlisle, PA",GET,Home,1540006905796.0,514,None,200,1542069417796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796.0,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,None,Logged In,Maia,F,0,Burke,None,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1540676534796.0,510,None,200,1542071524796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51


In [19]:
df_log.shape

(7302, 18)

In [20]:
# Is there any song intersection between datasets?
songs_intersection = set(df_log["song"]).intersection(set(df_songs["title"]))
songs_intersection

{'Intro', 'Setanta matins'}

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [21]:
# Filtering records by NextSong action, Home action doesnt have a song registered
df_log = df_log[df_log["page"]=="NextSong"]
df_log.reset_index(inplace=True, drop=True)

In [22]:
timestamp = pd.to_datetime(df_log["ts"], unit='ms').values
hour = pd.to_datetime(df_log["ts"], unit='ms').dt.hour.values
day = pd.to_datetime(df_log["ts"], unit='ms').dt.day.values
week = pd.to_datetime(df_log["ts"], unit='ms').dt.isocalendar().week.values
month = pd.to_datetime(df_log["ts"], unit='ms').dt.month.values
year = pd.to_datetime(df_log["ts"], unit='ms').dt.year.values
weekday = pd.to_datetime(df_log["ts"], unit='ms').dt.weekday.values

column_labels = ("start_time", "hour", "day", "week", "month", "year", "weekday")
time_data = (timestamp, hour, day, week, month, year, weekday)

df_time = pd.DataFrame(dict(zip(column_labels, time_data)))
df_time.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-13 00:40:37.796,0,13,46,11,2018,1
1,2018-11-13 01:12:29.796,1,13,46,11,2018,1
2,2018-11-13 03:19:02.796,3,13,46,11,2018,1
3,2018-11-13 03:51:52.796,3,13,46,11,2018,1
4,2018-11-13 05:00:06.796,5,13,46,11,2018,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
for i, row in df_time.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
df_user = df_log[["userId",
                  "firstName",
                  "lastName",
                  "gender",
                  "level"]]\
                .dropna(axis=0, subset=["userId"])

df_user["userId"] = df_user["userId"].astype(int)
df_user.head(1)

,userId,firstName,lastName,gender,level
0,66,Kevin,Arellano,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in df_user.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [26]:
for index, row in df_log.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    # insert songplay record
    timestamp = pd.to_datetime(row.ts, unit='ms')
    songplay_data = (timestamp,
                     row.userId,
                     row.level,
                     songid,
                     artistid,
                     row.sessionId,
                     row.location,
                     row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [27]:
cur.close()
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.